# Vgrid DGGS key features

You can try out Vgrid DGGS by using the cloud-computing platforms below without having to install anything on your computer. Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/00_intro.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/master)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD)

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, please use [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

## Install vgrid

In [ ]:
# %pip install vgrid --upgrade

## DGGS Conversion
### latlon2dggs

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2h3

lat = 10.775276
lon = 106.706797
res = 10
h3_id = latlon2h3(lat, lon, res)
h3_id

### DGGS to Shapely Polygon

In [ ]:
from vgrid.conversion.dggs2geo.h32geo import h32geo
h3_geo = h32geo([h3_id])
h3_geo

### DGGS to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.h32geo import h32geojson

h3_geojson = h32geojson(h3_id)
h3_geojson

### Vector to DGGS

In [ ]:
from vgrid.conversion.vector2dggs.vector2rhealpix import vector2rhealpix

# Read GeoJSON from URL
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon.geojson"
)
vector_to_rhealpix = vector2rhealpix(file_path, resolution=10, compact=False, output_format="gpd")
vector_to_rhealpix.explore(tiles="CartoDB Positron")

### DGGS Compact

In [ ]:
from vgrid.conversion.dggscompact import rhealpixcompact
rhealpix_compacted = rhealpixcompact(vector_to_rhealpix, rhealpix_id="rhealpix", output_format="gpd")
rhealpix_compacted.plot(edgecolor="white")

#### DGGS Expand

In [ ]:
from vgrid.conversion.dggscompact import rhealpixexpand
rhealpix_expanded = rhealpixexpand(vector_to_rhealpix, resolution=11, 
                                rhealpix_id="rhealpix", output_format="gpd")
rhealpix_expanded.plot(edgecolor="white")

### DGGS Binning

In [ ]:
from vgrid.binning.s2bin import s2bin
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats = "count"
s2_bin = s2bin(file_path, resolution=16, stats=stats, output_format="gpd")
s2_bin.plot(
    column=stats,        # numeric column to base the colors on
    cmap='Spectral_r',        # color scheme (matplotlib colormap)
    legend=True,  
    linewidth=0.2          # boundary width (optional)
)

### Raster to DGGS

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.raster2dggs.raster2tilecode import raster2tilecode
from vgrid.utils.io import download_file

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)

raster_to_tilecode = raster2tilecode(raster_file, output_format="gpd")

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

tilecode_layer = folium.GeoJson(
    raster_to_tilecode,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 1,
    },
    popup=folium.GeoJsonPopup(
        fields=["tilecode", "band_1", "band_2", "band_3"],
        aliases=["Tilecode ID", "Band 1", "Band 2", "Band 3"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(tilecode_layer.get_bounds())

# Display the map
m

## DGGS Generator

In [ ]:
from vgrid.generator.h3grid import h3grid

h3_grid = h3grid(resolution=0,output_format="gpd")
h3_grid.plot(edgecolor='white')

### DGGS Statistics

In [ ]:
from vgrid.stats.h3stats import h3stats
h3stats()